In [1]:
def normalize(X_train, X_test):

    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))

    X_train = (X_train - mean) / std
    X_test = (X_test - mean) / std

    return X_train, X_test

In [2]:
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt
# from augment import augment
import os
from keras.datasets.cifar100 import load_data
from keras.utils import to_categorical



Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar100.load_data(label_mode='fine')
x_train, x_test = normalize(x_train, x_test)
# y_train = to_categorical(y_train, 100)
# y_test = to_categorical(y_test, 100)

seed = 777
np.random.seed(seed)
np.random.shuffle(x_train)
np.random.seed(seed)
np.random.shuffle(y_train)

In [12]:
learning_rate = 0.005
num_epochs = 30
batch_size = 150

In [5]:
num_display = 100

def get_model(X, by, is_reuse):
    inputs = X
    with tf.variable_scope('first'):
        conv1 = tf.layers.conv2d(X, 32, 3, padding='same',
                                kernel_initializer=tf.initializers.truncated_normal(stddev=0.02),) # (None, 32, 32, 128)
        bn1 = tf.layers.batch_normalization(conv1,
                                            name='bn1',
                                            reuse=tf.AUTO_REUSE)
        h1 = tf.nn.relu(bn1)
        maxpool1 = tf.layers.max_pooling2d(h1, 2, 2) # (None, 16, 16, 128)
        dropout1 = tf.layers.dropout(maxpool1, rate = 0.5,training=training)
        
    with tf.variable_scope('second'):
        conv2 = tf.layers.conv2d(dropout1, 64, 3, padding='same',
                               kernel_initializer=tf.initializers.truncated_normal(stddev=0.02))
        bn2 = tf.layers.batch_normalization(conv2,
                                            name='bn2',
                                            reuse=tf.AUTO_REUSE)
        h2 = tf.nn.relu(bn2)
        maxpool2 = tf.layers.max_pooling2d(h2, 2, 2) # (None, 7, 7, 256)
#         outs += outs 
    with tf.variable_scope('third'):
        conv3 = tf.layers.conv2d(maxpool2, 128, 3, padding='same',
                               kernel_initializer=tf.initializers.truncated_normal(stddev=0.02))
        bn3 = tf.layers.batch_normalization(conv3,
                                            name='bn3',
                                            reuse=tf.AUTO_REUSE)
        h3 = tf.nn.relu(bn3)
        maxpool3 = tf.layers.max_pooling2d(h3, 2, 2) # (None, 3, 3, 64)
        dropout3 = tf.layers.dropout(maxpool3, 0.5, training=training)
        
    with tf.variable_scope('fourth'):
#         outs += outs 
        conv4 = tf.layers.conv2d(dropout3, 256, 3, padding='same',
                               kernel_initializer=tf.initializers.truncated_normal(stddev=0.02)) # (None, 28, 28, 128)

        bn4 = tf.layers.batch_normalization(conv4)
        h4 = tf.nn.relu(bn4)
        maxpool4 = tf.layers.max_pooling2d(h4, 2, 2) # (None, 14, 14, 128)
        dropout4 = tf.layers.dropout(maxpool4, 0.5 ,training=training)
        
    with tf.variable_scope('fifth'):
        conv5 = tf.layers.conv2d(dropout4, 512, 3, padding='same',
                               kernel_initializer=tf.initializers.truncated_normal(stddev=0.02))
        bn5 = tf.layers.batch_normalization(conv5)
        h5 = tf.nn.relu(bn5)
        maxpool5 = tf.layers.max_pooling2d(h5, 2, 2) # (None, 7, 7, 256)
        
    flat = tf.reshape(maxpool5, (-1, maxpool5.shape[1]*maxpool5.shape[2]*maxpool5.shape[3]))
    with tf.variable_scope('dense'):
        dense1 = tf.layers.dense(flat, 386,
                              kernel_initializer=tf.initializers.truncated_normal(stddev=0.02))
        dense_h = tf.nn.relu(dense1)
        dense_dropout = tf.layers.dropout(dense_h,0.5)
        outputs = tf.layers.dense(dense_h, 100)
    softmax = tf.nn.softmax(outputs)
    one_hot = tf.squeeze(tf.one_hot(by, 100),axis=1)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, 
                                                      labels=one_hot))
    opt = tf.train.AdamOptimizer(learning_rate=learning_rate,).minimize(loss)

    preds = tf.argmax(softmax, axis=1)
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.squeeze(by, axis=1), preds), tf.float32))
    init = tf.global_variables_initializer()
    return {
        'loss': loss,
        'opt': opt,
        'preds': preds,
        'acc': acc,
        'init': init,
        'softmax' : softmax
    }

In [6]:
X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3],name = 'input_data')
by = tf.placeholder(tf.int64, name ='y_input_data')
training = tf.placeholder(tf.bool, name = 'training_bool')
model = get_model(X, by, True)

In [13]:
with tf.Session() as sess:
    sess.run(model['init'])
    for ind_epoch in range(0, num_epochs):
        print('Current iteration {}'.format(ind_epoch + 1))
        
        for ind_ in range(0, int(50000 / batch_size)):
            batch_X = x_train[ind_*batch_size:(ind_+1)*batch_size]
            batch_by = y_train[ind_*batch_size:(ind_+1)*batch_size]
            _, cur_loss, cur_acc = sess.run(
                [model['opt'], model['loss'], model['acc']],
                feed_dict={X: batch_X, by: batch_by, training :True})
#             print(sess.run(model['preds'], feed_dict={X:x_train[:10], training:False}))
#             if ind_ % num_display == 0:
        print('loss {0:.4f} acc {1:.4f}'.format(cur_loss, cur_acc))
    cur_acc_all = 0.0
    cur_loss_all = 0.0
    for ind_ in range(0, 10):
        cur_loss, cur_acc = sess.run(
                    [model['loss'], model['acc']],
                    feed_dict={X: x_test[ind_*1000:(ind_+1)*1000], 
                               by: y_test[ind_*1000:(ind_+1)*1000],
                              training : False})
        cur_loss_all += cur_loss
        cur_acc_all += cur_acc
    print('TEST: loss {0:.4f} acc {1:.4f}'.format(cur_loss_all / 10.0, 
                                              cur_acc_all / 10.0))

Current iteration 1
loss 4.1856 acc 0.0533
Current iteration 2
loss 3.9122 acc 0.1133
Current iteration 3
loss 3.6389 acc 0.1333
Current iteration 4
loss 3.4629 acc 0.1933
Current iteration 5
loss 3.2745 acc 0.2200
Current iteration 6
loss 3.1841 acc 0.2200
Current iteration 7
loss 3.0797 acc 0.2533
Current iteration 8
loss 3.0206 acc 0.2933
Current iteration 9
loss 2.8793 acc 0.2533
Current iteration 10
loss 2.7345 acc 0.3333
Current iteration 11
loss 2.7995 acc 0.3467
Current iteration 12
loss 2.7544 acc 0.3200
Current iteration 13
loss 2.7639 acc 0.3333
Current iteration 14
loss 2.6846 acc 0.3333
Current iteration 15
loss 2.7106 acc 0.3533
Current iteration 16
loss 2.5576 acc 0.3333
Current iteration 17
loss 2.6235 acc 0.3467
Current iteration 18
loss 2.6128 acc 0.3667
Current iteration 19
loss 2.4458 acc 0.3600
Current iteration 20
loss 2.4118 acc 0.3400
Current iteration 21
loss 2.4754 acc 0.3733
Current iteration 22
loss 2.4182 acc 0.3667
Current iteration 23
loss 2.4925 acc 0.37

In [6]:
sess = tf.InteractiveSession()
tf.arg_max(tf.squeeze(tf.one_hot(y_train[:10], 100), axis=1),1).eval()

Instructions for updating:
Use `argmax` instead


array([90, 38, 99, 22, 55, 79, 86, 37, 92, 81], dtype=int64)

In [9]:
sess = tf.Session()
sess.run(model['init'])
sess.run(model['preds'], feed_dict={X:x_train[:5], training:False})

array([85, 85, 85, 85, 62], dtype=int64)

In [8]:
np.squeeze(y_train[:5], axis=1)

array([90, 38, 99, 22, 55])